In [5]:
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
import pandas as pd
import numpy as np

#train_data_source = './exercise_data/human_dna_train_small.csv'
train_data_source = './exercise_data/human_dna_train_split_5_95.csv'
val_data_source = './exercise_data/human_dna_validation_split.csv'
test_data_source = './exercise_data/human_dna_test_split.csv'

train_df = pd.read_csv(train_data_source, header=0)
test_df = pd.read_csv(test_data_source, header=0)

train_seq = train_df['sequences']
train_label = train_df['labels']
test_seq = test_df['sequences']
test_label = test_df['labels']

# Preprocess
tk = text.Tokenizer(char_level=True)
tk.fit_on_texts(train_seq)

train_seq_tok = tk.texts_to_sequences(train_seq)
test_seq_tok = tk.texts_to_sequences(test_seq)

train_seq = np.array(train_seq)
train_seq_tok = np.array(train_seq_tok)
train_label = np.array(train_label)
test_seq = np.array(test_seq)
test_seq_tok = np.array(test_seq_tok)
test_label = np.array(test_label)

print('train_seq shape:', train_seq.shape)
print('test_seq shape:', test_seq.shape)

Using TensorFlow backend.


train_seq shape: (29420,)
test_seq shape: (33333,)


In [6]:
train_label[train_label<0]=0
test_label[test_label<0]=0

In [7]:
train_label

array([1, 1, 1, ..., 0, 0, 0])

In [8]:
train_seq_tok = train_seq_tok - np.ones(shape=train_seq_tok.shape)
test_seq_tok = test_seq_tok - np.ones(shape=test_seq_tok.shape)

In [11]:
train_seq_tok

array([[0., 3., 1., ..., 1., 0., 1.],
       [0., 2., 2., ..., 0., 0., 0.],
       [1., 2., 2., ..., 1., 2., 0.],
       ...,
       [2., 0., 0., ..., 1., 1., 2.],
       [1., 3., 1., ..., 2., 1., 1.],
       [2., 0., 0., ..., 1., 1., 1.]])

In [13]:
train_seq_tok_split = np.hsplit(train_seq_tok, 2)

In [15]:
train_seq_tok.shape

(29420, 398)

In [19]:
train_seq_tok_split[0].shape

(29420, 199)

In [20]:
train_seq_tok[0]

array([0., 3., 1., 1., 2., 1., 2., 2., 0., 0., 1., 1., 2., 3., 1., 1., 3.,
       0., 0., 2., 0., 2., 2., 1., 0., 2., 2., 0., 3., 1., 0., 2., 2., 1.,
       3., 2., 1., 0., 1., 2., 2., 2., 1., 1., 3., 0., 1., 0., 0., 2., 3.,
       1., 1., 1., 0., 0., 3., 0., 0., 2., 1., 2., 3., 1., 0., 2., 2., 0.,
       2., 0., 0., 2., 1., 3., 1., 2., 2., 2., 1., 1., 2., 1., 1., 1., 1.,
       3., 1., 2., 0., 1., 0., 0., 3., 3., 1., 2., 1., 0., 3., 1., 2., 0.,
       0., 1., 2., 0., 2., 0., 2., 2., 3., 1., 2., 0., 1., 1., 1., 1., 3.,
       1., 3., 1., 0., 2., 1., 2., 2., 2., 2., 0., 2., 1., 3., 0., 2., 1.,
       1., 2., 2., 2., 1., 1., 2., 3., 1., 2., 3., 1., 1., 1., 1., 1., 3.,
       0., 2., 2., 3., 1., 2., 1., 1., 1., 1., 1., 1., 3., 1., 1., 2., 3.,
       1., 2., 3., 0., 3., 0., 0., 0., 3., 0., 3., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 2., 3., 3., 1., 1., 1., 2., 2., 1., 0., 1., 3.,
       3., 0., 3., 3., 3., 0., 3., 1., 1., 3., 1., 1., 0., 2., 2., 2., 3.,
       1., 0., 0., 3., 3.

In [23]:
test_seq_tok_split = np.hsplit(test_seq_tok, 2)

In [22]:
train_seq_tok_split[0][0]

array([0., 3., 1., 1., 2., 1., 2., 2., 0., 0., 1., 1., 2., 3., 1., 1., 3.,
       0., 0., 2., 0., 2., 2., 1., 0., 2., 2., 0., 3., 1., 0., 2., 2., 1.,
       3., 2., 1., 0., 1., 2., 2., 2., 1., 1., 3., 0., 1., 0., 0., 2., 3.,
       1., 1., 1., 0., 0., 3., 0., 0., 2., 1., 2., 3., 1., 0., 2., 2., 0.,
       2., 0., 0., 2., 1., 3., 1., 2., 2., 2., 1., 1., 2., 1., 1., 1., 1.,
       3., 1., 2., 0., 1., 0., 0., 3., 3., 1., 2., 1., 0., 3., 1., 2., 0.,
       0., 1., 2., 0., 2., 0., 2., 2., 3., 1., 2., 0., 1., 1., 1., 1., 3.,
       1., 3., 1., 0., 2., 1., 2., 2., 2., 2., 0., 2., 1., 3., 0., 2., 1.,
       1., 2., 2., 2., 1., 1., 2., 3., 1., 2., 3., 1., 1., 1., 1., 1., 3.,
       0., 2., 2., 3., 1., 2., 1., 1., 1., 1., 1., 1., 3., 1., 1., 2., 3.,
       1., 2., 3., 0., 3., 0., 0., 0., 3., 0., 3., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 2., 3., 3., 1., 1., 1., 2.])

In [30]:
from keras.layers import Dropout, concatenate, Input
import keras as K

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

# max_features = 20000
batch_size = 32
epochs = 100
learning_rate = 0.001


# First Half
first_half = Input(shape=(199,))
x = Dense(150, activation='relu', input_shape=(199,))(first_half)
x = Dropout(0.2)(x)
x = Dense(100, activation='relu', input_shape=(150,))(x)
x = Dropout(0.2)(x)
x = Dense(75, activation='relu', input_shape=(100,))(x)
x = Dropout(0.2)(x)
x = Dense(50, activation='relu', input_shape=(75,))(x)
out_first = Dropout(0.2)(x)

second_half = Input(shape=(199,))
x = Dense(150, activation='relu', input_shape=(199,))(second_half)
x = Dropout(0.2)(x)
x = Dense(100, activation='relu', input_shape=(150,))(x)
x = Dropout(0.2)(x)
x = Dense(75, activation='relu', input_shape=(100,))(x)
x = Dropout(0.2)(x)
x = Dense(50, activation='relu', input_shape=(75,))(x)
out_second = Dropout(0.2)(x)

concatenated = concatenate([out_first, out_second])
x = Dense(100, activation='relu')(concatenated)
# model.add(Dropout(0.2))
x = Dense(50, activation='relu')(x)
# model.add(Dropout(0.2))
x = Dense(20, activation='relu')(x)
# model.add(Dropout(0.2))
out_final = Dense(1, activation='sigmoid')(x)

model = K.Model([first_half, second_half], out_final)

# try using different optimizers and different optimizer configs
opt = K.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

# model.add(Dense(150, activation='relu', input_shape=(198,)))
# model.add(Dropout(0.2))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(75, activation='relu'))
# model.add(Dropout(0.2))
# out_a = model.add(Dense(50, activation='relu'))

# # Second Half
# model.add(Dense(150, activation='relu', input_shape=(198,)))
# model.add(Dropout(0.2))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(75, activation='relu'))
# model.add(Dropout(0.2))
# out_b = model.add(Dense(50, activation='relu'))

# # Concatenate
# concatenated = concatenate([out_a, out_b])
# model.add(Dense(100, activation='relu')(concatenated))
# # model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu'))
# # model.add(Dropout(0.2))
# model.add(Dense(20, activation='relu'))
# # model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))

# # try using different optimizers and different optimizer configs
# opt = K.optimizers.Adam(learning_rate=learning_rate)
# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

print('Train...')
model.fit(train_seq_tok_split, train_label, batch_size=batch_size, epochs=epochs, validation_data=(test_seq_tok_split, test_label))
score, acc = model.evaluate(test_seq_tok_split, test_label, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 29420 samples, validate on 33333 samples
Epoch 1/100
29420/29420 [==============================] - 7s 244us/step - loss: 0.2133 - acc: 0.9499 - val_loss: 0.1422 - val_acc: 0.9969
Epoch 2/100
29420/29420 [==============================] - 6s 214us/step - loss: 0.1985 - acc: 0.9500 - val_loss: 0.0678 - val_acc: 0.9969
Epoch 3/100
29420/29420 [==============================] - 6s 215us/step - loss: 0.1894 - acc: 0.9500 - val_loss: 0.0784 - val_acc: 0.9969
Epoch 4/100
29420/29420 [==============================] - 6s 211us/step - loss: 0.1823 - acc: 0.9500 - val_loss: 0.0643 - val_acc: 0.9969
Epoch 5/100
29420/29420 [==============================] - 6s 215us/step - loss: 0.1770 - acc: 0.9500 - val_loss: 0.0363 - val_acc: 0.9969
Epoch 6/100
29420/29420 [==============================] - 6s 209us/step - loss: 0.1710 - acc: 0.9500 - val_loss: 0.0451 - val_acc: 0.9969
Epoch 7/100
29420/29420 [==============================] - 6s 210us/step - loss: 0.1649 - acc: 0.9501 - val

29420/29420 [==============================] - 6s 208us/step - loss: 0.0254 - acc: 0.9910 - val_loss: 0.0546 - val_acc: 0.9863
Epoch 60/100
29420/29420 [==============================] - 6s 210us/step - loss: 0.0309 - acc: 0.9886 - val_loss: 0.0616 - val_acc: 0.9880
Epoch 61/100
29420/29420 [==============================] - 6s 211us/step - loss: 0.0306 - acc: 0.9901 - val_loss: 0.0470 - val_acc: 0.9923
Epoch 62/100
29420/29420 [==============================] - 6s 213us/step - loss: 0.0275 - acc: 0.9898 - val_loss: 0.0606 - val_acc: 0.9863
Epoch 63/100
29420/29420 [==============================] - 6s 213us/step - loss: 0.0249 - acc: 0.9907 - val_loss: 0.0557 - val_acc: 0.9893
Epoch 64/100
29420/29420 [==============================] - 6s 209us/step - loss: 0.0242 - acc: 0.9911 - val_loss: 0.0669 - val_acc: 0.9892
Epoch 65/100
29420/29420 [==============================] - 6s 211us/step - loss: 0.0264 - acc: 0.9910 - val_loss: 0.0490 - val_acc: 0.9899
Epoch 66/100
29420/29420 [=======